In [2]:
import os
import io
from IPython.display import Image,display, HTML
from PIL import Image
import base64
from transformers import pipeline, AutoTokenizer
import json
import torch
device = torch.device("cpu")  # Force CPU usage



In [3]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Read environment variables
hf_token = os.getenv("HF_API_TOKEN")

In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch


# Define the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# Correct model name
model_name = "sshleifer/distilbart-cnn-12-6"

# Load tokenizer & model with authentication
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, token=hf_token).to(device)

# Initialize pipeline correctly
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=0 if device.type != "cpu" else -1  # Use GPU/MPS if available
)

Device set to use mps:0


In [5]:
def summarize_text(input_text: str) -> dict:
    """Summarizes the given text.

    Args:
        input_text (str): The input text to be summarized.

    Returns:
        dict: A dictionary containing the summary.
    """
    # Ensure input text is within tokenizer's max length
    max_length = min(1024, tokenizer.model_max_length)  # Some models have higher limits

    inputs = tokenizer(
        input_text, 
        max_length=max_length, 
        return_tensors="pt", 
        truncation=True
    ).to(device)  # Move to correct device

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"], 
        num_beams=4, 
        max_length=100, 
        early_stopping=True
    )

    # Decode output
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return {"summary": summary_text}

# Example Usage
# print(summarize_text("Hugging Face provides thousands of NLP models for text summarization."))

In [6]:
text = """ These are short, famous texts in English from classic sources like the Bible or Shakespeare. 
Some texts have word definitions and explanations to help you. 
Some of these texts are written in an old style of English. 
Try to understand them, because the English that we speak today is based on what our great, great, great, great grandparents spoke before! 
Of course, not all these texts were originally written in English. 
The Bible, for example, is a translation. But they are all well known in English today, and many of them express beautiful thoughts."""

summarize_text(text)

# print (summarize_text(text))



{'summary': ' These are short, famous texts in English from classic sources like the Bible or Shakespeare . Some of these texts are written in an old style of English . Not all these texts were originally written in English . But they are all well known in English today, and many express beautiful thoughts .'}

In [12]:
import os
import gradio as gr

# Set default port safely
PORT = int(os.environ.get("PORT", 7860))

def generate_summary(input_text):
    output = summarize_text(input_text)
    return output

gr.close_all()
demo = gr.Interface(
    fn=generate_summary,
    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
    outputs=[gr.Textbox(label="Summary", lines=3)],
    title="Text Summarization",
    description="Summarize text using a pre-trained model."
)

demo.launch(share=True, server_port=PORT)


* Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://73aa849598240353b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
